<a href="https://colab.research.google.com/github/AlfredMoore/music_genre_transformer_classification/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Googlle Colab Git

from google.colab import drive
import os
root_path = "/content/drive/MyDrive/Umich_Study/EECS545/Final_Project"
drive.mount('/content/drive')

os.chdir(root_path)
# check pwd
! pwd
# python version
! python3 --version

Mounted at /content/drive
/content/drive/MyDrive/Umich_Study/EECS545/Final_Project
Python 3.9.16


In [ ]:
# root_path = "/content/drive/MyDrive/Umich_Study/EECS545/Final_Project"
# code_path = os.path.join(root_path,"music_genre_classification/code")
# metadata_path = os.path.join(root_path,"music_genre_classification/dataset/fma_metadata")
# fma_medium_path = os.path.join(root_path, "music_genre_classification/dataset/fma_medium/fma_medium")


In [ ]:
# Git Clone
os.chdir(root_path)
!git clone https://github.com/CVxTz/music_genre_classification.git
!git clone https://github.com/mdeff/fma.git
!mkdir music_genre_classification/dataset
!ls

fatal: destination path 'music_genre_classification' already exists and is not an empty directory.
fatal: destination path 'fma' already exists and is not an empty directory.
dataset  data_test.ipynb  fma  music_genre_classification  Project.ipynb


In [ ]:
# !pip install -r requirements.txt
# !pip install tensorflow tqdm scikit_learn
#  librosa numpy  pandas matplotlib
# !pip show librosa

In [ ]:
# download dataset

# !mkdir dataset
# !wget -O dataset/fma_metadata.zip -nc https://os.unil.cloud.switch.ch/fma/fma_metadata.zip 
# !wget -O dataset/fma_medium.zip -nc https://os.unil.cloud.switch.ch/fma/fma_medium.zip
# !wget -O dataset/fma_large.zip -nc https://os.unil.cloud.switch.ch/fma/fma_large.zip
# !ls dataset

In [ ]:
# unzip
# !unzip -n fma_metadata.zip
# !ls fma_metadata

# failed

In [ ]:
!pwd
!ls /content/drive/MyDrive/Umich_Study/EECS545/Final_Project/music_genre_classification/dataset

/content/drive/MyDrive/Umich_Study/EECS545/Final_Project
fma_medium  fma_metadata


*1*. Prepare Dataset
=====

  fma_metadata

In [ ]:
os.chdir( os.path.join(root_path,"music_genre_classification/code") )
import audio_processing
import prepare_data
# from music_genre_classification.code import audio_processing
# from music_genre_classification.code import prepare_data

from zipfile import ZipFile
import json
import numpy as np
import ast
import sys
import warnings
import pandas as pd

if not sys.warnoptions:
    warnings.simplefilter("ignore")


In [ ]:
# Prepare Data

# load metadata

metadata_path = os.path.join(root_path,"music_genre_classification/dataset/fma_metadata")
# zip_path = os.path.join(dataset_path,"fma_metadata.zip")
# zf = ZipFile(zip_path)
# print(zf.filelist)

df = prepare_data.load( os.path.join(metadata_path,"tracks.csv") )
df2 = pd.read_csv( os.path.join(metadata_path,"genres.csv") )

out_path = os.path.join(metadata_path,"tracks_genre.json")
mapping_path = os.path.join(metadata_path,"mapping.json")

# in_path = "/media/ml/data_ml/fma_metadata/tracks.csv"
# genres_path = "/media/ml/data_ml/fma_metadata/genres.csv"
# df = load("/media/ml/data_ml/fma_metadata/tracks.csv")
# df2 = pd.read_csv(genres_path)


In [ ]:

id_to_title = {k: v for k, v in zip(df2.genre_id.tolist(), df2.title.tolist())}
df.reset_index(inplace=True)

In [ ]:
# df.shape
# id_to_title

In [ ]:
# print(df.head())
# print(df.columns.values)
print(set(df[("set", "subset")].tolist()))

{'medium', 'large', 'small'}


In [ ]:
df = df[df[("set", "subset")].isin(["small", "medium", "large"])]
print(
  df[
      [
          ("track_id", ""),
          ("track", "genre_top"),
          ("track", "genres"),
          ("set", "subset"),
      ]
  ]
)

       track_id     track                    set
                genre_top         genres  subset
0             2   Hip-Hop           [21]   small
1             3   Hip-Hop           [21]  medium
2             5   Hip-Hop           [21]   small
3            10       Pop           [10]   small
4            20       NaN      [76, 103]   large
...         ...       ...            ...     ...
106569   155316      Rock           [25]   large
106570   155317      Rock           [25]   large
106571   155318      Rock           [25]   large
106572   155319      Rock           [25]   large
106573   155320       NaN  [10, 12, 169]   large

[106574 rows x 4 columns]


In [ ]:
data = {
  k: [id_to_title[a] for a in v]
  for k, v in zip(df[("track_id", "")].tolist(), df[("track", "genres")].tolist())
}
# data

In [ ]:
out_path = os.path.join(metadata_path,"tracks_genre.json")
mapping_path = os.path.join(metadata_path,"mapping.json")

json.dump(data, open(out_path, "w"), indent=4)
mapping = {k: i for i, k in enumerate(df2.title.tolist())}
json.dump(mapping, open(mapping_path, "w"), indent=4)

*2*. Audio Processing
======

track errors list:

001486.mp3, 005574.mp3, 065753.mp3, 108925.mp3, 143992.mp3, 133297.mp3, 098571.mp3, 105247.mp3, 126981.mp3, 080391.mp3, 098559.mp3, 098558.mp3, 098560.mp3, 099134.mp3

they should be removed

In [ ]:
# unzip
# !unzip -n /content/drive/MyDrive/Umich_Study/EECS545/Final_Project/dataset/fma_medium.zip -d /content/drive/MyDrive/Umich_Study/EECS545/Final_Project/music_genre_classification/dataset/fma_medium

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Umich_Study/EECS545/Final_Project/music_genre_classification/dataset/fma_medium")
!echo "# extracted"
!find . -type f | wc -l
!echo "# expected extracted   -14"
!zipinfo /content/drive/MyDrive/Umich_Study/EECS545/Final_Project/dataset/fma_medium.zip | grep files

root_path = "/content/drive/MyDrive/Umich_Study/EECS545/Final_Project"
code_path = os.path.join(root_path,"music_genre_classification/code")
metadata_path = os.path.join(root_path,"music_genre_classification/dataset/fma_metadata")
fma_medium_path = os.path.join(root_path, "music_genre_classification/dataset/fma_medium/fma_medium")

os.chdir(root_path)


# extracted
49972
# expected extracted   -14
25002 files, 24815301654 bytes uncompressed, 23821584112 bytes compressed:  4.0%


In [ ]:
# TODO modify
!du -h --max-depth=1 /content/drive/MyDrive/Umich_Study/EECS545/Final_Project/music_genre_classification/dataset/fma_medium

35G	/content/drive/MyDrive/Umich_Study/EECS545/Final_Project/music_genre_classification/dataset/fma_medium/fma_medium
35G	/content/drive/MyDrive/Umich_Study/EECS545/Final_Project/music_genre_classification/dataset/fma_medium


In [ ]:
import librosa

from music_genre_classification.code import audio_processing
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from multiprocessing import Pool

input_length = 16000 * 30
n_mels = 128

In [ ]:
## delete allow_tickle==Flase data
# files_transed = sorted(list(glob(fma_medium_path + "/*/*.npy")))
# for file in files_transed:
#   os.remove(file)

In [ ]:

files = sorted(list(glob(fma_medium_path + "/*/*.mp3")))
files_transed = sorted(list(glob(fma_medium_path + "/*/*.npy")))

for i in range(len(files_transed)):
  files_transed[i] = files_transed[i].replace(".npy",".mp3")

try:
  print(files_transed[0])
except:
  print(len(files_transed))

remained_files = list(set(files) - set(files_transed))

print("# tracks ",len(files))
print("# files_transed",len(files_transed))
print("# remaining tracks ",len(remained_files))
assert len(files)-len(files_transed)==len(remained_files)

# p = Pool(8)
# for i, _ in tqdm(enumerate(p.imap(audio_processing.save, files))):
#   if i % 1000 == 0:
#     print(i)

0
# tracks  24985
# files_transed 0
# remaining tracks  24985


In [ ]:
print(remained_files)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
p = Pool(8)
for i, _ in tqdm(enumerate(p.imap(audio_processing.save, remained_files))):
    if i % 1000 == 0:
        print(i)

3it [00:50, 13.01s/it]

0


963it [02:41, 12.43it/s]/content/drive/MyDrive/Umich_Study/EECS545/Final_Project/music_genre_classification/code/audio_processing.py:25: UserWarning: PySoundFile failed. Trying audioread instead.
  data = librosa.core.load(file_path, sr=16000)[0]  # , sr=16000
/usr/local/lib/python3.9/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
1000it [02:46,  6.85it/s]

1000


2001it [04:45,  8.95it/s]

...

23002it [46:59, 10.70it/s]

23000


24001it [49:00,  9.88it/s]

24000


24985it [51:00,  8.16it/s]


In [ ]:
# !pip show librosa numpy scikit_learn numba tensorflow

*3*. Pretrain Transformer
=====

In [ ]:
import os
os.chdir( os.path.join(root_path,"music_genre_classification/code") )

import json
from glob import glob

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from models import transformer_pretrain
from prepare_data import get_id_from_path, PretrainGenerator

root_path = "/content/drive/MyDrive/Umich_Study/EECS545/Final_Project"
code_path = os.path.join(root_path,"music_genre_classification/code")
metadata_path = os.path.join(root_path,"music_genre_classification/dataset/fma_metadata")
fma_medium_path = os.path.join(root_path, "music_genre_classification/dataset/fma_medium/fma_medium")



In [ ]:
from collections import Counter

h5_name = "transformer_pretrain.h5"
batch_size = 32
epochs = 100
id_to_genres = json.load(open( out_path ))
id_to_genres = {int(k): v for k, v in id_to_genres.items()}

base_path = fma_medium_path     # fma medium dataset
files = sorted(list(glob(base_path + "/*/*.npy")))
files = [x for x in files if id_to_genres[int(get_id_from_path(x))]]
labels = [id_to_genres[int(get_id_from_path(x))] for x in files]
print(len(labels))

samples = list(zip(files, labels))

strat = [a[-1] for a in labels]
cnt = Counter(strat)
strat = [a if cnt[a] > 2 else "" for a in strat]

train, val = train_test_split(
    samples, test_size=0.2, random_state=1337, stratify=strat
)

model = transformer_pretrain()

try:
    model.load_weights(h5_name, by_name=True)
    print("Weights Loaded")
except:
    print("Could not load weights")

checkpoint = ModelCheckpoint(
    h5_name,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    mode="min",
    save_weights_only=True,
)
reduce_o_p = ReduceLROnPlateau(
    monitor="val_loss", patience=20, min_lr=1e-7, mode="min"
)

try:
    model.load_weights(h5_name)
except:
    print("Could not load weights")

model.fit_generator(
    PretrainGenerator(train, batch_size=batch_size),
    validation_data=PretrainGenerator(val, batch_size=batch_size),
    epochs=epochs,
    callbacks=[checkpoint, reduce_o_p],
    use_multiprocessing=True,
    workers=12,
    verbose=2,
    max_queue_size=64,
)

*4*. Transformer Claasifier
=====

In [ ]:
import json
from glob import glob

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from models import transformer_classifier
from prepare_data import get_id_from_path, DataGenerator


In [ ]:
from collections import Counter

h5_name = "transformer.h5"
batch_size = 32
epochs = 50
CLASS_MAPPING = json.load(open( mapping_path ))
id_to_genres = json.load(open( out_path ))
id_to_genres = {int(k): v for k, v in id_to_genres.items()}

base_path = fma_medium_path
files = sorted(list(glob(base_path + "/*/*.npy")))
files = [x for x in files if id_to_genres[int(get_id_from_path(x))]]
labels = [id_to_genres[int(get_id_from_path(x))] for x in files]
print(len(labels))

samples = list(zip(files, labels))

strat = [a[-1] for a in labels]
cnt = Counter(strat)
strat = [a if cnt[a] > 2 else "" for a in strat]

train, val = train_test_split(
    samples, test_size=0.2, random_state=1337, stratify=strat
)

model = transformer_classifier(n_classes=len(CLASS_MAPPING))

checkpoint = ModelCheckpoint(
    h5_name,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    mode="min",
    save_weights_only=True,
)
reduce_o_p = ReduceLROnPlateau(
    monitor="val_loss", patience=20, min_lr=1e-7, mode="min"
)

model.fit_generator(
    DataGenerator(train, batch_size=batch_size, class_mapping=CLASS_MAPPING),
    validation_data=DataGenerator(
        val, batch_size=batch_size, class_mapping=CLASS_MAPPING
    ),
    epochs=epochs,
    callbacks=[checkpoint, reduce_o_p],
    use_multiprocessing=True,
    workers=12,
    verbose=2,
    max_queue_size=64,
)

In [ ]:
import numpy as np
a = np.load("/content/drive/MyDrive/Umich_Study/EECS545/Final_Project/music_genre_classification/dataset/fma_medium/fma_medium/000/000002.npy")
